In [87]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from pynsys.nsys_scanner import NsysScanner, fix_chunks

 # Compare results on RTX 3050Ti vs RTX 2080 Ti vs A100
 
 
A100_path = "../data/compare_gpu/a100/"
RTX2080TI_path = "../data/compare_gpu/rtx2080ti/"
RTX3050TI_path = "../data/compare_gpu/rtx3050ti/"

GPU_A100 = 'A100'
GPU_RTX2080TI = 'RTX2080TI'
GPU_RTX3050TI = 'RTX3050TI'

FILE_SIZE = 524288000




In [88]:
A100_result_df = fix_chunks(pd.read_csv(A100_path + '/results.csv'), FILE_SIZE)
RTX2080TI_result_df = fix_chunks(pd.read_csv(RTX2080TI_path + '/results.csv'), FILE_SIZE)
RTX3050TI_result_df = fix_chunks(pd.read_csv(RTX3050TI_path + '/results.csv'), FILE_SIZE)

A100_result_df['gpu'] = GPU_A100
RTX2080TI_result_df['gpu'] = GPU_RTX2080TI
RTX3050TI_result_df['gpu'] = GPU_RTX3050TI


all_result_df = pd.concat([A100_result_df, RTX2080TI_result_df, RTX3050TI_result_df])

all_result_df = all_result_df.rename(columns={'Standard': 'standard', 'File': 'file', 'chunksize_Bytes': 'chunk_size'})

all_result_df = all_result_df.sort_values(['gpu', 'standard', 'file', 'chunk_size'])

all_result_df = all_result_df[~all_result_df['file'].str.contains("lineitem.parquet")]
all_result_df = all_result_df[~all_result_df['file'].str.contains("silesia.zip")]


all_result_df


,standard,file,chunks,decompression_throughput_GBps,compression_throughput_GBps,ratio,chunk_size,chunksize_Bytes_formatted,chunks_formatted,chunks_combined_formatted,gpu
128,bitcomp,500mb_dickens,131072,66.7744,123.9354,1.1863,4000.0,3.90625kB,128kB,128kB chunks of 3.90625kBB,A100
129,bitcomp,500mb_dickens,32768,74.7685,182.9488,1.2040,16000.0,15.625kB,32kB,32kB chunks of 15.625kBB,A100
130,bitcomp,500mb_dickens,8192,46.6345,144.3853,1.2077,64000.0,62.5kB,8kB,8kB chunks of 62.5kBB,A100
131,bitcomp,500mb_dickens,2048,16.7542,58.4108,1.2087,256000.0,250kB,2kB,2kB chunks of 250kBB,A100
132,bitcomp,500mb_dickens,512,4.7090,15.9800,1.2090,1024000.0,1000kB,512B,512B chunks of 1000kBB,A100
...,...,...,...,...,...,...,...,...,...,...,...
104,zstd,500mb_xml,2048,0.4522,0.1020,6.5364,256000.0,250kB,2kB,2kB chunks of 250kBB,RTX3050TI
105,zstd,500mb_xml,512,0.2050,0.0722,6.7145,1024000.0,1000kB,512B,512B chunks of 1000kBB,RTX3050TI
106,zstd,500mb_xml,128,0.0770,0.0372,6.9958,4096000.0,3.90625MB,128B,128B chunks of 3.90625MBB,RTX3050TI
107,zstd,500mb_xml,32,0.0550,0.0198,6.8314,16384000.0,15.625MB,32B,32B chunks of 15.625MBB,RTX3050TI


In [89]:
compressors = ['snappy', 'zstd', 'gdeflate', 'lz4', 'cascaded', 'bitcomp']
# compression_files = ['500mb_dickens', '500mb_mozilla', '500mb_mr', '500mb_nci', '500mb_samba', '500mb_sao', '500mb_silesia.zip', '500mb_xml', '500mb_lineitem.parquet']
compression_files = ['500mb_dickens', '500mb_mozilla', '500mb_mr', '500mb_nci', '500mb_samba', '500mb_sao', '500mb_xml']

approx_number_of_threads = [8, 32, 128, 512, 2048, 8192, 32768, 131072]


A100_utilisation_df = NsysScanner(A100_path, FILE_SIZE).get_utilisation_df(compression_files, compressors, approx_number_of_threads)
RTX2080TI_utilisation_df = NsysScanner(RTX2080TI_path, FILE_SIZE).get_utilisation_df(compression_files, compressors, approx_number_of_threads)
RTX3050TI_utilisation_df = NsysScanner(RTX3050TI_path, FILE_SIZE).get_utilisation_df(compression_files, compressors, approx_number_of_threads)

A100_utilisation_df['gpu'] = GPU_A100
RTX2080TI_utilisation_df['gpu'] = GPU_RTX2080TI
RTX3050TI_utilisation_df['gpu'] = GPU_RTX3050TI


all_utilization_df = pd.concat([A100_utilisation_df, RTX2080TI_utilisation_df, RTX3050TI_utilisation_df])

all_utilization_df = all_utilization_df.sort_values(['gpu', 'standard', 'file', 'chunk_size'])

all_df = pd.merge(all_result_df, all_utilization_df, on=['gpu', 'standard', 'file', 'chunk_size'])

all_df

500mb_dickens  snappy  8  1.0  1.0
500mb_dickens  snappy  32  1.0  1.0
500mb_dickens  snappy  128  1.0  1.0
500mb_dickens  snappy  512  1.0  1.0
500mb_dickens  snappy  2048  1.0  1.0
500mb_dickens  snappy  8192  4.6598186471508445  4.592107155622175
500mb_dickens  snappy  32768  16.446449571398187  15.40027323174332
500mb_dickens  snappy  131072  30.534915522398933  49.53148608637113
500mb_dickens  zstd  8  0.9999985991308262  0.9999920661830579
500mb_dickens  zstd  32  0.999998687185812  0.999992068638917
500mb_dickens  zstd  128  0.9999608183726257  0.9999758329190052
500mb_dickens  zstd  512  0.9999186947109855  0.9985239773564485
500mb_dickens  zstd  2048  0.9998750248338928  1.8040136326033442
500mb_dickens  zstd  8192  1.956758141039657  2.8352382279703923
500mb_dickens  zstd  32768  7.981332503494909  10.121186740289875
500mb_dickens  zstd  131072  26.831776814488247  24.462600266628012
Failed getting utilization of gdeflate 500mb_dickens 8 threads: "Unable to synchronously open

,standard,file,chunks,decompression_throughput_GBps,compression_throughput_GBps,ratio,chunk_size,chunksize_Bytes_formatted,chunks_formatted,chunks_combined_formatted,gpu,compression_utilization,decompression_utilization
0,bitcomp,500mb_dickens,131072,66.7744,123.9354,1.1863,4000.0,3.90625kB,128kB,128kB chunks of 3.90625kBB,A100,4.000000,7.506414
1,bitcomp,500mb_dickens,32768,74.7685,182.9488,1.2040,16000.0,15.625kB,32kB,32kB chunks of 15.625kBB,A100,0.000000,10.580370
2,bitcomp,500mb_dickens,8192,46.6345,144.3853,1.2077,64000.0,62.5kB,8kB,8kB chunks of 62.5kBB,A100,6.000000,8.607039
3,bitcomp,500mb_dickens,2048,16.7542,58.4108,1.2087,256000.0,250kB,2kB,2kB chunks of 250kBB,A100,1.500000,3.942393
4,bitcomp,500mb_dickens,512,4.7090,15.9800,1.2090,1024000.0,1000kB,512B,512B chunks of 1000kBB,A100,1.000000,0.993999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,zstd,500mb_xml,2048,0.4522,0.1020,6.5364,256000.0,250kB,2kB,2kB chunks of 250kBB,RTX3050TI,1.560409,3.569931
809,zstd,500mb_xml,512,0.2050,0.0722,6.7145,1024000.0,1000kB,512B,512B chunks of 1000kBB,RTX3050TI,0.999785,1.391390
810,zstd,500mb_xml,128,0.0770,0.0372,6.9958,4096000.0,3.90625MB,128B,128B chunks of 3.90625MBB,RTX3050TI,0.999761,0.999951
811,zstd,500mb_xml,32,0.0550,0.0198,6.8314,16384000.0,15.625MB,32B,32B chunks of 15.625MBB,RTX3050TI,0.999916,0.999967


In [90]:
fig_gpus_througphut_compression = px.line(
    all_result_df,
    title=f"Compression throughput", 
    x="chunk_size",
    y="compression_throughput_GBps",
    color="file",
    log_x=True,
    # log_y=True,
    markers=True,
    facet_row='standard',
    facet_col="gpu",
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_gpus_througphut_compression.update_xaxes(autorange="reversed")
# fig_gpus_througphut_compression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_througphut_compression.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_gpus_througphut_compression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)

fig_gpus_througphut_compression.update_layout(
    yaxis10_title="Compression throughput (GB/s)",
    xaxis_title="",
    xaxis2_title="Chunk size",
    xaxis3_title="",
)

fig_gpus_througphut_compression.show()

In [91]:
fig_gpus_througphut_decompression = px.line(
    all_result_df,
    title=f"Decompression throughput", 
    x="chunk_size",
    y="decompression_throughput_GBps",
    color="file",
    log_x=True,
    markers=True,
    # log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_gpus_througphut_decompression.update_xaxes(autorange="reversed")
# fig_gpus_througphut_decompression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_througphut_decompression.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_gpus_througphut_decompression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)

fig_gpus_througphut_decompression.update_layout(
    yaxis12_title="Decompression throughput (GB/s)",
    xaxis_title="",
    xaxis2_title="Chunk size",
    xaxis3_title="",
)


fig_gpus_througphut_decompression.show()

In [92]:
# Compression ratio
fig_gpus_ratio= px.line(
    all_result_df,
    title=f"Compression ratio", 
    x="chunk_size",
    y="ratio",
    color="file",
    log_x=True,
    markers=True,
    # log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_gpus_througphut_decompression.update_xaxes(autorange="reversed")
# fig_gpus_througphut_decompression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_ratio.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_gpus_ratio.update_yaxes(matches=match_y, row=row + 1, col=col + 1)

fig_gpus_ratio.update_layout(
    yaxis12_title="Compression ratio",
    xaxis_title="",
    xaxis2_title="Chunk size",
    xaxis3_title="",
)

fig_gpus_ratio.show()

In [93]:
fig_gpus_ratio_compression= px.line(
    all_result_df,
    title=f"Compression throughput per compression ratio", 
    x="ratio",
    y="compression_throughput_GBps",
    color="file",
    log_x=True,
    markers=True,
    # log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_gpus_througphut_decompression.update_xaxes(autorange="reversed")
# fig_gpus_througphut_decompression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_ratio_compression.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_gpus_ratio_compression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)

fig_gpus_ratio_compression.update_layout(
    yaxis10_title="Compression throughput (GB/s)",
    xaxis2_title="Compression ratio",
)

fig_gpus_ratio_compression.show()

In [94]:
fig_gpus_ratio_decompression= px.line(
    all_result_df,
    title=f"Decompression throughput per compression ratio", 
    x="ratio",
    y="decompression_throughput_GBps",
    color="file",
    log_x=True,
    markers=True,
    # log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_gpus_througphut_decompression.update_xaxes(autorange="reversed")
# fig_gpus_througphut_decompression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_ratio_decompression.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_gpus_ratio_decompression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)

fig_gpus_ratio_decompression.update_layout(
    yaxis10_title="Decompression throughput (GB/s)",
    xaxis2_title="Compression ratio",
)

fig_gpus_ratio_decompression.show()

In [95]:
fig_gpus_utilization_compression = px.line(
    all_utilization_df,
    title=f"GPU utilization compression", 
    x="chunk_size",
    y="compression_utilization",
    color="file",
    log_x=True,
    markers=True,
    # log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_gpus_utilization_compression.update_xaxes(autorange="reversed")
# fig_gpus_utilization_compression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_utilization_compression.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_gpus_utilization_compression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)

fig_gpus_utilization_compression.update_layout(
    yaxis12_title="Utilization (%)",
    xaxis_title="",
    xaxis2_title="Chunk size",
    xaxis3_title="",
)

fig_gpus_utilization_compression.show()

In [96]:
fig_gpus_utilization_decompression = px.line(
    all_utilization_df,
    title=f"GPU utilization decompression", 
    x="chunk_size",
    y="decompression_utilization",
    color="file",
    log_x=True,
    markers=True,
    # log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_gpus_utilization_decompression.update_xaxes(autorange="reversed")
# fig_gpus_utilization_decompression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_gpus_utilization_decompression.update_yaxes(title=None)

fig_gpus_utilization_decompression.update_layout(
    yaxis12_title="Utilization (%)",
    xaxis_title="",
    xaxis2_title="Chunk size",
    xaxis3_title="",
)

fig_gpus_utilization_decompression.show()

In [97]:
# compression utilization vs throughput

fig_utilization_throughput_compression = px.scatter(
    all_df,
    title=f"Compression utilization vs throughput", 
    x="compression_utilization",
    y="compression_throughput_GBps",
    color="file",
    log_x=True,
    # markers=True,
    #log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_utilization_throughput_compression.update_xaxes(autorange="reversed")
# fig_utilization_throughput_compression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_utilization_throughput_compression.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_utilization_throughput_compression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)
        
        
fig_utilization_throughput_compression.update_layout(
    yaxis10_title="Compression hroughput (GB/s)",
    xaxis_title="",
    xaxis2_title="Utilization (%)",
    xaxis3_title="",
)

fig_utilization_throughput_compression.show()

In [98]:
# Decompression utilization vs throughput

fig_utilization_throughput_decompression = px.scatter(
    all_df,
    title=f"Decompression utilization vs throughput", 
    x="decompression_utilization",
    y="decompression_throughput_GBps",
    color="file",
    log_x=True,
    # markers=True,
    #log_y=True,
    facet_col='gpu',
    facet_row='standard',
    facet_col_spacing=0.05,
    width=1200,
    height=1000,
)
# fig_utilization_throughput_compression.update_xaxes(autorange="reversed")
# fig_utilization_throughput_compression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_utilization_throughput_decompression.update_yaxes(title=None,matches=None,showticklabels=True)


nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_utilization_throughput_decompression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)
        
        
fig_utilization_throughput_decompression.update_layout(
    yaxis10_title="Decompression hroughput (GB/s)",
    xaxis_title="",
    xaxis2_title="Utilization (%)",
    xaxis3_title="",
)

fig_utilization_throughput_decompression.show()

In [176]:
# Compression throughput per utilization

all_df['compression_throughput_per_utilization'] = all_df['compression_throughput_GBps'] / all_df['compression_utilization']

all_df = all_df[all_df['compression_throughput_per_utilization'] >= 0]
# all_df = all_df.sort_values(by=['chunk_size'])

all_df["chunk_size_str"] = all_df["chunk_size"].astype(int).astype(str)

fig_utilization_throughput_compression = px.bar(all_df, 
                                                title=f"Compression throughput per utilization", 
                                                x='chunk_size_str', 
                                                y='compression_throughput_per_utilization',
                                                color="file",
                                                hover_data=['chunk_size'],
                                                barmode='group',
                                                # log_x=True,
                                                # markers=True,
                                                # log_y=True,
                                                facet_col='gpu',
                                                facet_row='standard',
                                                facet_col_spacing=0.05,
                                                facet_row_spacing=0.07,
                                                width=1200,
                                                height=1000,
                                                category_orders={"gpu": ["A100", "RTX2080TI", "RTX3050TI"]}
                                                )
# px.scatter(
#     all_df,
#     title=f"Compression utilization vs throughput", 
#     y="compression_throughput_per_utilization",
#     x="compression_utilization",
#     color="file",
#     log_x=True,
#     # markers=True,
#     #log_y=True,
#     facet_col='gpu',
#     facet_row='standard',
#     facet_col_spacing=0.05,
#     width=1200,
#     height=1000,
# )
# fig_utilization_throughput_compression.update_xaxes(autorange="reversed")
# fig_utilization_throughput_compression.update_xaxes(dict(
#         tickmode = 'array',
#         tickvals = alldf['chunksize_Bytes'],
#         ticktext = alldf['chunksize_Bytes_formatted'],
#         tickangle=45,
#     )
# )
fig_utilization_throughput_compression.update_yaxes(title=None,matches=None,showticklabels=True)
fig_utilization_throughput_compression.update_xaxes(title=None,showticklabels=True)

nb_rows = 6
nb_cols = 3
for row in range(nb_rows):
    match_y = "y" if row == 0 else f"y{row * nb_cols + 1}"
    for col in range(nb_cols):
        fig_utilization_throughput_compression.update_yaxes(matches=match_y, row=row + 1, col=col + 1)
        
        
# fig_utilization_throughput_compression.update_layout(
#     yaxis10_title="Throughput per utilization (GB/s/%)",
#     xaxis_title="",
#     xaxis2_title="",
#     xaxis3_title="",
# )

fig_utilization_throughput_compression.show()

all_df


,standard,file,chunks,decompression_throughput_GBps,compression_throughput_GBps,ratio,chunk_size,chunksize_Bytes_formatted,chunks_formatted,chunks_combined_formatted,gpu,compression_utilization,decompression_utilization,compression_throughput_per_utilization,chunk_size_str
0,bitcomp,500mb_dickens,131072,66.7744,123.9354,1.1863,4000.0,3.90625kB,128kB,128kB chunks of 3.90625kBB,A100,4.000000,7.506414,30.983850,4000
32,bitcomp,500mb_samba,131072,95.0450,139.7351,1.2083,4000.0,3.90625kB,128kB,128kB chunks of 3.90625kBB,A100,19.000000,19.779474,7.354479,4000
590,bitcomp,500mb_samba,131072,28.1169,31.0237,1.2083,4000.0,3.90625kB,128kB,128kB chunks of 3.90625kBB,RTX3050TI,43.833328,74.756920,0.707765,4000
582,bitcomp,500mb_nci,131072,29.3603,28.8418,1.5250,4000.0,3.90625kB,128kB,128kB chunks of 3.90625kBB,RTX3050TI,46.672726,77.737238,0.617958,4000
574,bitcomp,500mb_mr,131072,31.1798,30.8007,1.7534,4000.0,3.90625kB,128kB,128kB chunks of 3.90625kBB,RTX3050TI,35.534688,51.209192,0.866778,4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,snappy,500mb_dickens,8,0.0174,0.0054,1.4984,65536000.0,62.5MB,8B,8B chunks of 62.5MBB,A100,1.000000,1.000000,0.005400,65536000
782,zstd,500mb_mr,8,0.0399,0.0105,2.6925,65536000.0,62.5MB,8B,8B chunks of 62.5MBB,RTX3050TI,0.999996,0.999987,0.010500,65536000
31,bitcomp,500mb_nci,8,0.6049,1.5462,1.5563,65536000.0,62.5MB,8B,8B chunks of 62.5MBB,A100,1.000000,0.999741,1.546200,65536000
703,lz4,500mb_sao,8,0.0324,0.0094,1.1224,65536000.0,62.5MB,8B,8B chunks of 62.5MBB,RTX3050TI,0.999945,1.000000,0.009401,65536000
